In [50]:
import sys
import os
import pandas as pd
import numpy as np
sys.path.insert(0, r"K:/Thesis/codes/crypto_project")
sys.path.insert(0, r"E:/Thesis/crypto_project")

In [2]:
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "database_server.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django

django.setup()

In [35]:
from crypto_calculator.models import RawPriceData
from  crypto_calculator.serializers import RawPriceDataSerializer
from crypto_calculator.models import FactorReturns
from crypto_calculator.models import Exposures


In [51]:
from crypto_calculator.sample_risk_input import portfolio_details, market_portfolio, risk_calculation_parameters

from factor_model.risk_calculations.core_universe_portfolio import generate_market_portfolio
from crypto_calculator.risk_calc_request_processing import get_factor_return_data, get_exposures, get_specific_returns
from factor_model.risk_calculations.factor_covariance import generate_factor_covariance_matrix
from factor_model.risk_calculations.risk_attribution import generate_active_space_portfolio
from factor_model.risk_calculations.risk_attribution import calculate_spec_risk_mctr, create_portfolio_exposures, generate_factor_covariance_attribution, generate_factor_covariance_table, get_specific_risk_beta
from factor_model.risk_calculations.risk_metrics import calculate_lognormal_es_var
from factor_model.risk_calculations.specific_risk import generate_raw_portfolio_specific_risk, generate_raw_specific_risk
from factor_model.model_update.database_generators import EXPOSURE_NON_STYLE_FIELDS



In [52]:
# Step 1: cob_date and basic queries
cob_date = risk_calculation_parameters["date"]
exposures = get_exposures(cob_date=cob_date)

if market_portfolio is None:
    market_portfolio = generate_market_portfolio(exposures)

all_tickers = list(set(market_portfolio.keys()).union(set(portfolio_details.keys())))
full_specific_returns = get_specific_returns(cob_date=cob_date, symbols=all_tickers)
factor_returns = get_factor_return_data(cob_date=cob_date)

# Step 2: calculate risk
factor_covariance = generate_factor_covariance_matrix(factor_returns, risk_calculation_parameters)
active_space_port = generate_active_space_portfolio(portfolio_details, market_portfolio)

portolios = {
    "portfolio": portfolio_details,
    "market": market_portfolio,
    "active": active_space_port,
}

port_exposures = {}
factor_risks = {}
factor_attributions = {}
factor_covars = {}
raw_specific_risks = {}
spec_risk_availabilities = {}
spec_risks = {}
total_risks = {}
factor_mctrs = {}
spec_risk_attributions = {}
spec_risk_var_decomps = {}
spec_risk_mctrs = {}
for port in portolios.keys():
    # 1. exposure calc
    port_exposures[port] = create_portfolio_exposures(
        exposures=exposures, portfolio_details=portolios[port], non_style_fields=EXPOSURE_NON_STYLE_FIELDS
    )
    port_exposures[port] = port_exposures[port].reindex(factor_covariance.index)
    # 2. factor risk related
    (
        factor_risks[port],
        factor_attributions[port],
    ) = generate_factor_covariance_attribution(port_exposures[port], factor_covariance)
    factor_covars[port] = generate_factor_covariance_table(
        port_exposures[port], factor_covariance
    )
    # 3. spec risk related
    (
        raw_specific_risks[port],
        spec_risk_availabilities[port],
    ) = generate_raw_specific_risk(
        full_specific_returns, risk_calculation_parameters, portolios[port]
    )
    spec_risks[port] = generate_raw_portfolio_specific_risk(raw_specific_risks[port], portolios[port])
    total_risks[port] = np.sqrt(factor_risks[port]**2 + spec_risks[port]**2)
    
    factor_mctrs[port] = factor_attributions[port]/total_risks[port]
    spec_risk_attributions[port], spec_risk_var_decomps[port] = calculate_spec_risk_mctr(raw_specific_risks[port], portolios[port])
    spec_risk_mctrs[port] = spec_risk_attributions[port] / total_risks[port]
    
# 4. beta calculation...
factor_beta_covar, _ = generate_factor_covariance_attribution(port_exposures["portfolio"], factor_covariance, port_exposures["market"])
spec_risk_covar = get_specific_risk_beta(portolios["portfolio"], market_portfolio=portolios["market"], spec_risk= raw_specific_risks["active"])
portfolio_beta = (factor_beta_covar**2 + spec_risk_covar)/(total_risks["market"]**2)

# 5. expected shortfall calculation assuming normality, 1 Day
es95, var95 = calculate_lognormal_es_var(total_risks["portfolio"], 0.95)
es99, var99 = calculate_lognormal_es_var(total_risks["portfolio"], 0.99)


In [69]:
# 6. assemble output
risk_categories = ['Total Risk (portfolio)', "Total Risk (market)", "Total Risk (active)",
              'Factor Risk (portfolio)', 'Factor Risk (active)',
              'Specific Risk (portfolio)', 'Specific Risk (active)']
risk_values = [total_risks["portfolio"]*100, total_risks["market"]*100, total_risks["active"]*100,
          factor_risks["portfolio"]*100, factor_risks["active"]*100,
          spec_risks["portfolio"]*100, spec_risks["active"]*100
          ]
risk_metrics = dict(zip(risk_categories, risk_values))

risk_metrics_extended = risk_metrics.copy()
risk_metrics_extended["portfolio Beta"] = portfolio_beta*100
risk_metrics_extended["portfolio VaR (1-day, 95%, total space)"] = var95*100
risk_metrics_extended["portfolio ES (1-day, 95%, total space)"] = es95*100
risk_metrics_extended["portfolio VaR (1-day, 99%, total space)"] = var99*100
risk_metrics_extended["portfolio ES (1-day, 99%, total space)"] = es99*100
for key in risk_metrics_extended.keys():
    risk_metrics_extended[key] = np.round(risk_metrics_extended[key], decimals=3)

In [70]:
risk_metrics_extended

{'Total Risk (portfolio)': 4.253,
 'Total Risk (market)': 3.683,
 'Total Risk (active)': 12.151,
 'Factor Risk (portfolio)': 3.339,
 'Factor Risk (active)': 5.682,
 'Specific Risk (portfolio)': 2.635,
 'Specific Risk (active)': 10.741,
 'portfolio Beta': 88.342,
 'portfolio VaR (1-day, 95%, total space)': 6.757,
 'portfolio ES (1-day, 95%, total space)': 8.388,
 'portfolio VaR (1-day, 99%, total space)': 9.421,
 'portfolio ES (1-day, 99%, total space)': 10.709}